In [ ]:
import subprocess
import os
import torchaudio
import torch
# import disp
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd

input_file = '/path/to/media.wav'
output_file = '/path/to/output.wav'

In [ ]:
import whisperx
models = {}
try:
	models['m1'] = whisperx.load_model(
			'large-v2',
			device='cuda',
			compute_type='float16',
			language='en'
		)
except:
	print('OOM?')

In [ ]:
import whisperx
try:
	models['m2'] = whisperx.load_model(
			'large-v2',
			device='cuda',
			compute_type='float16',
			language='en'
		)
except:
	print('OOM?')
	import gc, torch
	del models
	gc.collect()
	torch.cuda.empty_cache()

In [ ]:
# using Mayavoz
from mayavoz.models import Mayamodel

model = Mayamodel.from_pretrained("shahules786/mayavoz-waveunet-valentini-28spk")
assert isinstance(model, Mayamodel)

res = model.enhance(input_file) # type: ignore
res

In [ ]:
# using resemble-enhance

from resemble_enhance.enhancer.inference import denoise, enhance

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

path = input_file
solver = 'euler'
nfe = 50 # 1 to 128
denoising = True
lambd = 0.9 if denoising else 0.1
tau = 0.5

dwav, sr1 = torchaudio.load(path) # type: ignore
orig_wav = dwav.cpu().numpy()
dwav = dwav.mean(dim=0)
print(sr1)

wav1, sr2 = denoise(dwav, sr1, device)
wav2, sr3 = enhance(dwav, sr1, device, nfe=nfe, solver=solver, lambd=lambd, tau=tau)

# denoise works pretty well
# enhance damaged speech in my first test (maybe use less steps)

print(sr1, sr2, sr3)
wav1 = wav1.cpu().numpy()
wav2 = wav2.cpu().numpy()

ipd.display(ipd.Audio(orig_wav, rate=sr1))
ipd.display(ipd.Audio(wav1, rate=sr2))
ipd.display(ipd.Audio(wav2, rate=sr3))

# does denoiser take vram into account?
# my usage goes up to 11.64/12GB

# save denoised
# torchaudio.save(output_file, torch.tensor(wav1), new_sr) # type: ignore
